In [1]:
# HACK: use project root as the working directory 
from pathlib import Path

while Path.cwd().name != 'language-model-toxicity':
    %cd ..

/homes/gws/sgehman/language-model-toxicity


In [2]:
from typing import List
import logging
from pathlib import Path
import tempfile

from joblib import Memory, Parallel, delayed
import numpy as np
import scipy.sparse
from tqdm.auto import tqdm

from utils.constants import DATA_DIR, OUTPUT_DIR

# Create joblib memory
mem = Memory(OUTPUT_DIR / 'cache' / 'webtext_overlap')

In [3]:
EOS = 50256
vocab_size = EOS + 1

## Load vecs

In [4]:
owtc_vecs_file = OUTPUT_DIR / 'owtc_vecs.npz'
wt_vecs_file = OUTPUT_DIR / 'wt_vecs.npz'

assert owtc_vecs_file.exists() and wt_vecs_file.exists()

In [5]:
owtc_vecs = scipy.sparse.load_npz(owtc_vecs_file)

In [6]:
# wt_vecs = scipy.sparse.load_npz(wt_vecs_file)

## Use sklearn

In [ ]:
%load_ext autoreload
%autoreload 2

In [22]:
import pysparnn.cluster_index as ci

In [23]:
doc_ids = list(range(owtc_vecs.shape[0]))
cp = ci.ClusterIndex(owtc_vecs, doc_ids)

KeyboardInterrupt: 

In [10]:
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

In [11]:
ann_svd = make_pipeline(
    TruncatedSVD(n_components=30),
    NearestNeighbors(n_jobs=64),
    verbose=True
)

In [12]:
ann_svd.fit(owtc_vecs)

[Pipeline] ...... (step 1 of 2) Processing truncatedsvd, total=54.5min
[Pipeline] .. (step 2 of 2) Processing nearestneighbors, total= 3.7min


Pipeline(memory=None,
         steps=[('truncatedsvd',
                 TruncatedSVD(algorithm='randomized', n_components=30, n_iter=5,
                              random_state=None, tol=0.0)),
                ('nearestneighbors',
                 NearestNeighbors(algorithm='auto', leaf_size=30,
                                  metric='minkowski', metric_params=None,
                                  n_jobs=64, n_neighbors=5, p=2, radius=1.0))],
         verbose=True)

In [13]:
x = _

In [17]:
x

Pipeline(memory=None,
         steps=[('truncatedsvd',
                 TruncatedSVD(algorithm='randomized', n_components=30, n_iter=5,
                              random_state=None, tol=0.0)),
                ('nearestneighbors',
                 NearestNeighbors(algorithm='auto', leaf_size=30,
                                  metric='minkowski', metric_params=None,
                                  n_jobs=64, n_neighbors=5, p=2, radius=1.0))],
         verbose=True)

In [18]:
import pickle

In [19]:
from joblib import dump

In [21]:
dump(x['truncatedsvd'], 'truncatedsvd_owtc.joblib')

['truncatedsvd_owtc.joblib']

In [22]:
dump(x['nearestneighbors'], 'nearestneighbors_owtc.joblib')

['nearestneighbors_owtc.joblib']